In [ ]:
%%capture
%pip install aiofiles
%pip install py7zr
%pip install imagecodecs
%pip install boto3
%pip install nest_asyncio
%pip install multivolumefile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
import shutil
from pathlib import Path
import requests

import boto3
from botocore.config import Config

import asyncio
import aiohttp
import aiofiles

import nest_asyncio 
nest_asyncio.apply()

import zipfile
import py7zr
import multivolumefile

import uuid
import skimage

import numpy as np
import pandas as pd

from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio

In [3]:
class DatasetDownloader:
    def __init__(self, downloads:Path, urls:dict):

        # init required directories
        if not (downloads.exists() and downloads.is_dir()):
            downloads.mkdir(exist_ok=True, parents=True)
            print(f"download directory at {downloads}")
        self.download_dir = downloads

        # src_urls is a dictionary such that
        # src_urls[filename:str]  = url:str
        self.src_urls = urls
        
    async def async_download_one_file(self, session, url:str, file_path:Path):
        """Download one file from url and save to disk at file_path"""
        #TODO: How to use tqdm for each coroutine in the notebook
        async with session.get(url, ssl = False) as r:
            #total_size = int(r.headers.get('content-length', 0))
            async with aiofiles.open(file_path, "wb") as f:
                #progress_bar = tqdm(total=total_size, unit="B", unit_scale=True, desc="Downloading")
                async for chunk in r.content.iter_any():
                    await f.write(chunk)
                    #progress_bar.update(len(chunk))

    async def download_files(self, async_download_one_file = None) -> None:
        if not async_download_one_file:
            async_download_one_file = self.async_download_one_file
        #Download files from self.src_urls, skip if already_downloaded
        timeout = aiohttp.ClientTimeout(total = None)
        async with aiohttp.ClientSession(timeout=timeout, cookie_jar=aiohttp.CookieJar()) as session:
            coroutines = list()
            for file_name, url in self.src_urls.items():
                file_path = self.download_dir / file_name 
                coroutines.append(async_download_one_file(session, url, file_path))
            await tqdm_asyncio.gather(*coroutines)   
    
    def validate_download(self, downloaded_file_sizes: dict) -> None:
        #TODO: Implement Validate Downloads
        pass

In [4]:
class DatasetExtractor:

    @staticmethod
    def extract_zip_archive(zip_file_path:Path, target_dir:Path, dirs_to_be_extracted = list()):
        """Extract specified contents from zip archive, extract all contents if not specified"""

        with zipfile.ZipFile(zip_file_path, 'r') as zip:
            #If dirs_to_be_extracted is an empty list, extract entire archive and exit
            if not dirs_to_be_extracted:
                zip.extractall(target_dir); return
            #Otherwise, extract all files under specified dirs
            #For each file in archive, extract if it's under any specified dir
            for member in zip.infolist():
                for foldername in dirs_to_be_extracted:
                    if foldername in member.filename:
                        #TODO: Add tqdm progress bar for extraction
                        zip.extract(member, target_dir)

    @staticmethod    
    def extract_multivolume_archive(multivolume_file_path:Path, target_dir:Path) -> None:
        """Extract all contents of a multivolume 7zip archive""" 

        with multivolumefile.open(multivolume_file_path, mode = 'rb') as multi_archive:
            with py7zr.SevenZipFile(multi_archive, 'r') as archive: # type: ignore
                archive.extractall(path = target_dir)

    @staticmethod
    def validate_extraction(val_dir:Path, val_files: list):
        """Check if val_dir contains all files listed under val_files, return list of missing files"""
        pass

In [5]:
class CropUtil():

    def _read_image(self, path):
        return skimage.io.imread(path)  # type: ignore

    def _read_mask(self, path):
        return skimage.io.imread(path)  # type: ignore

    def _get_pad_amount(self, dimension: int, window: int):
        """Calculate to no of pixels to add to before and after dimension"""
        total_padding = window - (dimension % window)

        if total_padding % 2 == 0:
            after = total_padding // 2
            before = after
        else:
            after = (total_padding // 2) + 1
            before = after - 1
        assert before+after == total_padding 
        return (before, after)
    
    def _pad_3d_array(self, array: np.ndarray, window: int):
        """
        Pad image array s.t. divisible by window\n
        array.shape : (Height, Width, Channels)
        window : side length of square cropping window
        """

        assert array.ndim == 3
        padded_array = np.pad(
            array = array,
            pad_width = (self._get_pad_amount(array.shape[0], window),
                         self._get_pad_amount(array.shape[1], window),
                         (0, 0))
        ) 
        return padded_array
    
    def _get_cropped_view(self, array: np.ndarray, window:int):
        """
        Crop image array s.t. divisible by window\n
        array.shape : (Height, Width, Channels)
        window : side length of square cropping window
        """

        assert array.ndim == 3
        cropped_view = skimage.util.view_as_windows( # type: ignore
            arr_in = array,
            window_shape = (window, window, array.shape[2]),
            step =  (window, window, array.shape[2])).squeeze()
            
        cropped_view = cropped_view.reshape(-1, window, window, array.shape[2])

        return cropped_view

    def _crop_one_scene(self, tile_path: Path, window: int, read_scene):
        scene = read_scene(tile_path) 
        scene = self._pad_3d_array(scene, window)
        scene = self._get_cropped_view(scene, window)
        return scene

    def _save_as_jpeg_100(self, array: np.ndarray, out_path: Path) -> None:
        skimage.io.imsave((out_path.parent / f"{out_path.stem}.jpg"), array, check_contrast = False, **{"quality": 100}) # type: ignore

In [6]:
class ContaboStorage():
    def __init__(self, credentials_file: Path, bucket_name: str):

        assert (credentials_file.exists() and credentials_file.is_file())

        with open(credentials_file) as credentials:
            credentials = credentials.readlines()[1:]

        def get_key(x: str) ->str:
            return x.split('=')[-1].strip('\n').strip(' ')

        self.s3resource = boto3.resource("s3",
                          endpoint_url = "https://usc1.contabostorage.com",
                          aws_access_key_id = get_key(credentials[0]),
                          aws_secret_access_key = get_key(credentials[1]))

        self.bucket = self.s3resource.Bucket(bucket_name) # type: ignore
        self.prefix = {
            "train_image": Path("labelled/patches/images"),
            "train_mask": Path("labelled/patches/masks"),
            "train_catalog": Path("labelled/metadata/"),

            "test_image": Path("labelled/scenes/images"),
            "test_mask": Path("labelled/scenes/masks"),
            "test_catalog": Path("labelled/metadata/"),
        }

    
    def upload_file(self, file_path:Path, bucket_key:Path):
        assert file_path.exists() and file_path.is_file()
        bucket_obj = self.bucket.Object(bucket_key.as_posix())
        bucket_obj.upload_file(file_path)

    def download_file(self, bucket_key:Path, file_path:Path):
        bucket_obj = self.bucket.Object(bucket_key.as_posix())
        file_path.parent.mkdir(exist_ok=True, parents=True)
        bucket_obj.download_file(file_path)
    
    def delete_file(self, bucket_key:Path):
        #TODO: Finish This
        bucket_obj = self.bucket.Object(bucket_key.as_posix())
        #bucket_obj.delete_file(missing_ok = True)
    
    def download_train_catalog(self, csv_path: Path):
        self.download_file(self.prefix["train_catalog"] / "patches.csv", csv_path)

    def upload_train_catalog(self, csv_path: Path):
        self.upload_file(csv_path, self.prefix["train_catalog"] / "patches.csv")

    def download_test_catalog(self, csv_path: Path):
        self.download_file(self.prefix["test_catalog"] / "scenes.csv", csv_path)

    def upload_test_catalog(self, csv_path: Path):
        self.upload_file(csv_path, self.prefix["test_catalog"] / "scenes.csv")

    def upload_train_pair(self, file_name, image_dir, mask_dir):
        self.upload_file(file_path = (image_dir / file_name),
                         bucket_key = self.prefix["train_image"] / file_name)
        self.upload_file(file_path = (mask_dir / file_name),
                         bucket_key = self.prefix["train_mask"] / file_name)

    def upload_test_pair(self, file_name, image_dir, mask_dir):
        self.upload_file(file_path = (image_dir / file_name),
                         bucket_key = self.prefix["test_image"] / file_name)
        self.upload_file(file_path = (mask_dir / file_name),
                         bucket_key = self.prefix["test_mask"] / file_name)
   
    def delete_train_pair(self, file_name):
        pass
    
    def delete_test_pair(self, file_name):
        pass

    def list_files_in_bucket(self):
        for file_obj in self.bucket.objects.all():
            print(file_obj.key)

In [25]:
class S3StorageEmulator():
    def __init__(self, root: Path, credentials_file: Path, bucket_name: str):
        #specific to emulator
        root.mkdir(exist_ok = True, parents=True)

        assert (credentials_file.exists() and credentials_file.is_file())

        self.root = root 
        self.bucket = self.root / bucket_name

        self.prefix = {
            "train_image": Path("labelled/patches/images"),
            "train_mask": Path("labelled/patches/masks"),
            "train_catalog": Path("labelled/metadata/patches.csv"),

            "test_image": Path("labelled/scenes/images"),
            "test_mask": Path("labelled/scenes/masks"),
            "test_catalog": Path("labelled/metadata/scenes.csv"),
        }

    def upload_file(self, file_path:Path, bucket_key:Path):
        """Upload from local storage to configured bucket"""

        assert file_path.exists() and file_path.is_file(), "File not found on disk"
        storage_key = self.bucket / bucket_key

        #Only for Emulator, Make Sure Parent Directory Exists for Copying
        (storage_key.parent).mkdir(parents=True, exist_ok=True)

        #Emulator version of upload
        shutil.copy(file_path, storage_key)

    def download_file(self, bucket_key:Path, file_path: Path):
        """Download from configured bucket to local path in argument"""

        #Emulate error when file does not exist on bucket
        storage_key = self.bucket / bucket_key
        assert storage_key.exists() and storage_key.is_file(), "Key not found in bucket"
            
        #Ensure directory in which file_path is to be downloaded exists
        file_path.parent.mkdir(parents=True, exist_ok=True)

        #Emulator version of upload
        shutil.copy(storage_key, file_path)

    def delete_file(self, bucket_key):
        """Delete file from configured bucket"""
        storage_key = self.bucket / bucket_key
        assert storage_key.exists() and storage_key.is_file(), "Key not found in bucket"
        storage_key.unlink()

    def download_train_catalog(self, csv_path: Path):
        try:
            self.download_file(bucket_key = self.prefix["train_catalog"], 
                               file_path = csv_path)
        except AssertionError:
            print("Training Catalog Not Found")
            return
    
    def upload_train_catalog(self, csv_path: Path):
        self.upload_file(csv_path, self.prefix["train_catalog"])

    def upload_train_pair(self, file_name, image_dir, mask_dir):
        self.upload_file(file_path = (image_dir / file_name),
                         bucket_key = self.prefix["train_image"] / file_name)
        self.upload_file(file_path = (mask_dir / file_name),
                         bucket_key = self.prefix["train_mask"] / file_name)

    def upload_test_pair(self, file_name, image_dir, mask_dir):
        self.upload_file(file_path = (image_dir / file_name),
                         bucket_key = self.prefix["test_image"] / file_name)
        self.upload_file(file_path = (mask_dir / file_name),
                         bucket_key = self.prefix["test_mask"] / file_name)
    
    
    def delete_train_pair(self, file_name):
        self.delete_file(self.prefix["train_image"] / file_name)
        self.delete_file(self.prefix["train_mask"] / file_name)

    def delete_test_pair(self, file_name):
        self.delete_file(self.prefix["test_image"] / file_name)
        self.delete_file(self.prefix["test_mask"] / file_name)

    def list_files_in_bucket(self):
        for file_path in self.bucket.rglob("*"):
            print(file_path)


In [26]:
class InriaETL():

    urls = {
        "aerialimagelabeling.7z.001" : "https://files.inria.fr/aerialimagelabeling/aerialimagelabeling.7z.001",
        "aerialimagelabeling.7z.002" : "https://files.inria.fr/aerialimagelabeling/aerialimagelabeling.7z.002",
        "aerialimagelabeling.7z.003" : "https://files.inria.fr/aerialimagelabeling/aerialimagelabeling.7z.003",
        "aerialimagelabeling.7z.004" : "https://files.inria.fr/aerialimagelabeling/aerialimagelabeling.7z.004",
        "aerialimagelabeling.7z.005" : "https://files.inria.fr/aerialimagelabeling/aerialimagelabeling.7z.005"
    }

    sizes = {
        "aerialimagelabeling.7z.001" : 0,
        "aerialimagelabeling.7z.002" : 0,
        "aerialimagelabeling.7z.003" : 0,
        "aerialimagelabeling.7z.004" : 0,
        "aerialimagelabeling.7z.005" : 0
    }

    locations = ["austin", "chicago", "kitsap", "tyrol-w", "vienna"]
    test_files_list = [f"{location}{num}.tif" for location in locations for num in range(1, 7)]
    train_files_list = [f"{location}{num}.tif" for location in locations for num in range(7, 37)]

    def __init__(self, root: Path):
        self.root_dir = root
        self.download_dir = root / "downloads"

        # Downloaded Images, Masks
        self.d_dataset_dir = root / "AerialImageDataset" / "train"
        self.d_image_dir = self.d_dataset_dir / "images"
        self.d_mask_dir = self.d_dataset_dir / "gt"

        # Test Images, Masks
        self.t_image_dir = self.root_dir / "test" / "images"
        self.t_mask_dir = self.root_dir / "test" / "masks"

        #Images, Masks
        self.image_dir = self.root_dir / "images"
        self.mask_dir = self.root_dir / "masks"

    def download(self):
        downloader = DatasetDownloader(self.download_dir, self.urls)
        asyncio.run(downloader.download_files())
        #await asyncio.create_task(downloader.download_files())
        #downloader.validate_download()

    def extract(self, low_storage_mode:bool):
        extractor = DatasetExtractor()

        #Merge and Extract Dataset Zip
        multivolume_7zip_path = self.download_dir / "aerialimagelabeling.7z"
        dataset_zip_path = self.download_dir / "NEW2-AerialImageDataset.zip"

        extractor.extract_multivolume_archive(multivolume_7zip_path, self.download_dir)
        print(f"Extracted Multivolume Archive to {dataset_zip_path}")

        if low_storage_mode:
            print("Deleting downloaded multi-volume to save storage space")
            for volume in self.download_dir.glob("aerialimagelabeling.7z.*"):
                volume.unlink()

        extractor.extract_zip_archive(dataset_zip_path, self.root_dir, ["train"])
        print(f"Extracted Dataset Archive to {self.root_dir}")

        if low_storage_mode:
            print("Deleting extracted dataset archive to save storage space")
            dataset_zip_path.unlink()

        #extractor.validate_extraction(files_list, dataset_dir)

    def move_test_split(self):
        self.t_image_dir.mkdir(exist_ok=True, parents=True)
        self.t_mask_dir.mkdir(exist_ok=True, parents=True)

        for file_name in self.test_files_list:
            shutil.move((self.d_image_dir / file_name), self.t_image_dir)
            shutil.move((self.d_mask_dir / file_name), self.t_mask_dir)

    @staticmethod
    def read_mask(path:Path):
        return np.expand_dims(skimage.io.imread(path), -1) # type: ignore

    def crop(self, window: int):
        self.image_dir.mkdir(exist_ok=True)
        self.mask_dir.mkdir(exist_ok=True)

        cropper = CropUtil()

        for file_name in self.train_files_list:

            cropped_image_view = cropper._crop_one_scene(
                tile_path = self.d_image_dir / file_name,
                window = window,
                read_scene = cropper._read_image
            )

            cropped_mask_view = cropper._crop_one_scene(
                tile_path = self.d_mask_dir / file_name,
                window = window,
                read_scene = self.read_mask
            )

            for image_crop, mask_crop in zip(cropped_image_view, cropped_mask_view):
                crop_name = str(uuid.uuid4())
                cropper._save_as_jpeg_100(image_crop , (self.image_dir / crop_name))
                cropper._save_as_jpeg_100(mask_crop.squeeze(), (self.mask_dir / crop_name))

    def delete_downloads(self):
        shutil.rmtree(self.d_dataset_dir.parent)

    def upload_train_dataset(self, storage: ContaboStorage):
        train_files_list = list(self.mask_dir.glob("*.jpg")) 
        for file_path in tqdm(train_files_list):
            storage.upload_train_pair(file_path.name, self.image_dir, self.mask_dir)

    def upload_test_dataset(self, storage: ContaboStorage):
        test_files_list = list(self.mask_dir.glob("*.tif"))
        for file_path in tqdm(test_files_list):
            storage.upload_test_pair(file_path.name, self.t_image_dir, self.t_mask_dir)

    def catalog_train_dataset(self):
        """Returns a DataFrame with training patch names and dataset name as columns"""

        df = pd.DataFrame({"name": [x.name for x in self.image_dir.rglob("*.jpg")]}) 
        df["dataset"] = [self.root_dir.name]*len(df)
        return df

    def update_train_dataset(self, storage: ContaboStorage):

        #Set path for catalog to be downloaded to 
        #WARINING: training catalog path must be hamed "patches.csv"
        catalog_path = self.root_dir / "metadata" / "patches.csv"

        #If catalog file present in local storage, remove it
        catalog_path.unlink(missing_ok=True)
        catalog_path.parent.mkdir(exist_ok=True, parents=True)

        #Download catalog from bucket
        storage.download_train_catalog(catalog_path)

        #If a catalog is present in storage 
        if catalog_path.exists() and catalog_path.is_file():
            print("Downloaded Catalog Found")
            df = pd.read_csv(catalog_path)

            #Get a view of files associated with current dataset
            dataset_df = df[df["dataset"] == self.root_dir.name]

            #If any files in bucket are associated with the current dataset
            if len(dataset_df) > 0:
               #Remove those files from storage
                print(f"Deleting existing patches in bucket")
                existing_file_names = list(dataset_df.name.apply(lambda x: x))
                for file_name in tqdm(existing_file_names):
                    storage.delete_train_pair(file_name)
                
                #Remove those files from the catalog
                df = df.drop(dataset_df.index, axis = 0)

        
        #If catalog is not present in storage => dataset is not present in storage
        else:  
            #Create a new catalog placeholder
            print("Downloaded Catalog Not Found, Creating New")
            df = pd.DataFrame({"name" : list(), "dataset" : list()})
        
        #Upload training dataset to bucket
        print("Uploading patches to bucket")
        self.upload_train_dataset(storage)
        #Add patches in image, mask dir to catalog
        df = pd.concat([df, self.catalog_train_dataset()], axis = 0)
        #Save catalog to local storage
        df.to_csv(catalog_path, index = False)
        #Upload catalog to bucket
        storage.upload_train_catalog(catalog_path)
        catalog_path.unlink()

In [27]:
class CityOSMETL():
    urls = {
        "berlin.zip": "https://zenodo.org/record/1154821/files/berlin.zip?download=1",
        "chicago.zip": "https://zenodo.org/record/1154821/files/chicago.zip?download=1",
        "paris.zip": "https://zenodo.org/record/1154821/files/paris.zip?download=1",
        "potsdam.zip": "https://zenodo.org/record/1154821/files/potsdam.zip?download=1",
        "tokyo.zip": "https://zenodo.org/record/1154821/files/tokyo.zip?download=1",
        "zurich.zip": "https://zenodo.org/record/1154821/files/zurich.zip?download=1"
    } 

    def __init__(self, root: Path):
        self.root_dir = root
        self.download_dir = root / "downloads"

        self.d_image_dir = self.download_dir / "images"
        self.d_mask_dir = self.download_dir / "masks"

        self.image_dir = self.root_dir / "images"
        self.mask_dir = self.root_dir / "masks"

        self.t_image_dir= self.root_dir / "test" / "images"
        self.t_mask_dir= self.root_dir / "test" / "masks"

    def download(self):
        downloader = DatasetDownloader(self.download_dir, self.urls)
        asyncio.run(downloader.download_files())
    
    def _move_and_rename_files(self, file_paths) -> None:
        for file_path in file_paths:
            shutil.move(file_path, file_path.parents[1] / f"{file_path.stem.split('_')[0]}.png")
            
    def _extract_file_num(self, file_name: str):
        numbers = list()
        for char in file_name:
            if char.isdigit():
                numbers.append(char) 
        return int(''.join(numbers))   

    def extract(self, low_storage_space:bool):
        extractor = DatasetExtractor()

        print(f"Extracting downloaded archives")
        for zip_file_name in tqdm(self.urls.keys()):
            zip_file_path = self.download_dir / zip_file_name
            extractor.extract_zip_archive(zip_file_path, self.d_image_dir, ["image"])
            extractor.extract_zip_archive(zip_file_path, self.d_mask_dir, ["labels"])

        print(f"Reorganizing file structure")
        self._move_and_rename_files(self.d_image_dir.rglob("*.png"))
        self._move_and_rename_files(self.d_mask_dir.rglob("*.png"))

        if low_storage_space: 
            print(f"Deleting downloaded archives to save storage space")
            for zip_file_name in self.urls.keys():
                zip_file_path = self.download_dir / zip_file_name
                zip_file_path.unlink()
    
    def calculate_train_test_split(self):
        self.files_list = [x.name for x in self.d_image_dir.rglob("*.png")]
        location_numbers = dict()
        for file_name in self.files_list:
            location = file_name.split('.')[0].strip('1234567890')
            if location in location_numbers.keys():
                location_numbers[location].append(self._extract_file_num(file_name))
            else:
                location_numbers[location] = list()

        self.test_files_list = list()
        for location, numbers_list in location_numbers.items():
            for num in sorted(numbers_list)[:int(0.15*len(numbers_list))]:
                self.test_files_list.append(f"{location}{num}.png")
        
        self.train_files_list = list(set(self.files_list).difference(self.test_files_list))

    def move_test_split(self):
        self.t_image_dir.mkdir(exist_ok=True, parents=True)
        self.t_mask_dir.mkdir(exist_ok=True, parents=True)

        for file_name in self.test_files_list:
            shutil.move(self.d_image_dir / file_name, self.t_image_dir)
            shutil.move(self.d_mask_dir / file_name, self.t_mask_dir)
    
    @staticmethod
    def read_mask(path: Path):
        mask = skimage.io.imread(path)[:, :, 2] # type: ignore
        mask = np.where((mask == 0), np.uint8(255), np.uint8(0))
        return np.expand_dims(mask, -1)

    def crop(self, window: int):
        self.image_dir.mkdir(exist_ok=True)
        self.mask_dir.mkdir(exist_ok=True)

        cropper = CropUtil()

        for file_name in self.train_files_list:
            cropped_image_view = cropper._crop_one_scene(
                tile_path = self.d_image_dir / file_name,
                window = window,
                read_scene = cropper._read_image
            )

            cropped_mask_view = cropper._crop_one_scene(
                tile_path = self.d_mask_dir / file_name,
                window = window,
                read_scene = self.read_mask
            )

            for image_crop, mask_crop in zip(cropped_image_view, cropped_mask_view):
                crop_name = str(uuid.uuid4())
                cropper._save_as_jpeg_100(image_crop, (self.image_dir / crop_name))
                cropper._save_as_jpeg_100(mask_crop.squeeze(), (self.mask_dir / crop_name))
    
    def delete_downloads(self):
        shutil.rmtree(self.download_dir)

    def upload_train_dataset(self, storage: ContaboStorage):
        train_files_list = list(self.mask_dir.glob("*.jpg")) 
        for file_path in tqdm(train_files_list):
            storage.upload_train_pair(file_path.name, self.image_dir, self.mask_dir)

    def upload_test_dataset(self, storage: ContaboStorage):
        test_files_list = list(self.mask_dir.glob("*.tif"))
        for file_path in tqdm(test_files_list):
            storage.upload_test_pair(file_path.name, self.t_image_dir, self.t_mask_dir)

    def catalog_train_dataset(self):
        """Returns a DataFrame with training patch names and dataset name as columns"""

        df = pd.DataFrame({"name": [x.name for x in self.image_dir.rglob("*.jpg")]}) 
        df["dataset"] = [self.root_dir.name]*len(df)
        return df

    def update_train_dataset(self, storage: ContaboStorage):

        #Set path for catalog to be downloaded to 
        #WARINING: training catalog path must be hamed "patches.csv"
        catalog_path = self.root_dir / "metadata" / "patches.csv"

        #If catalog file present in local storage, remove it
        catalog_path.unlink(missing_ok=True)
        catalog_path.parent.mkdir(exist_ok=True, parents=True)

        #Download catalog from bucket
        storage.download_train_catalog(catalog_path)

        #If a catalog is present in storage 
        if catalog_path.exists() and catalog_path.is_file():
            print("Downloaded Catalog Found")
            df = pd.read_csv(catalog_path)

            #Get a view of files associated with current dataset
            dataset_df = df[df["dataset"] == self.root_dir.name]

            #If any files in bucket are associated with the current dataset
            if len(dataset_df) > 0:
               #Remove those files from storage
                print(f"Deleting existing patches in bucket")
                existing_file_names = list(dataset_df.name.apply(lambda x: x))
                for file_name in tqdm(existing_file_names):
                    storage.delete_train_pair(file_name)
                
                #Remove those files from the catalog
                df = df.drop(dataset_df.index, axis = 0)

        
        #If catalog is not present in storage => dataset is not present in storage
        else:  
            #Create a new catalog placeholder
            print("Downloaded Catalog Not Found, Creating New")
            df = pd.DataFrame({"name" : list(), "dataset" : list()})
        
        #Upload training dataset to bucket
        print("Uploading patches to bucket")
        self.upload_train_dataset(storage)
        #Add patches in image, mask dir to catalog
        df = pd.concat([df, self.catalog_train_dataset()], axis = 0)
        #Save catalog to local storage
        df.to_csv(catalog_path, index = False)
        #Upload catalog to bucket
        storage.upload_train_catalog(catalog_path)
        catalog_path.unlink()

In [28]:
class ISPRSETL():
    urls = {
        "potsdam.zip": "https://seafile.projekt.uni-hannover.de/f/429be50cc79d423ab6c4/",
        #"toronto.zip": "https://seafile.projekt.uni-hannover.de/f/fc62f9c20a8c4a34aea1/",
        "vaihingen.zip": "https://seafile.projekt.uni-hannover.de/f/6a06a837b1f349cfa749/",
    }
    password = "CjwcipT4-P8g"
    cookie_name = "sfcsrftoken"

    def __init__(self, root: Path):
        self.root_dir = root
        self.download_dir = root / "downloads"

        self.d_image_dir = self.download_dir / "images"
        self.d_mask_dir = self.download_dir / "masks"

        self.image_dir = self.root_dir / "images"
        self.mask_dir = self.root_dir / "masks"

        self.t_image_dir= self.root_dir / "test" / "images"
        self.t_mask_dir= self.root_dir / "test" / "masks"
    
    def download(self):
        downloader = DatasetDownloader(self.download_dir, self.urls)
        asyncio.run(downloader.download_files(self._download_file))

    def extract(self, low_storage_mode):
        extractor = DatasetExtractor 
        self._extract_vaihingen(extractor, low_storage_mode)
        self._extract_potsdam(extractor, low_storage_mode)
    
    def calculate_train_test_split(self):
        self.files_list = [x.name for x in self.d_image_dir.rglob("*.tif")]
        df = pd.DataFrame({"name": self.files_list})
        df["region"] = df.name.apply(lambda x:self._get_file_region(x))

        self.test_files_list = list()
        for region in df.region.unique():
            region_df = df[df["region"] == region]
            self.test_files_list += sorted(list(region_df.name))[:int(0.15*len(region_df))]
        
        self.train_files_list = sorted(list(set(self.files_list).difference(self.test_files_list)))
    
    def move_test_split(self):
        self.t_image_dir.mkdir(exist_ok=True, parents=True)
        self.t_mask_dir.mkdir(exist_ok=True, parents=True)

        for file_name in self.test_files_list:
            shutil.move((self.d_image_dir / file_name), self.t_image_dir)
            shutil.move((self.d_mask_dir / file_name), self.t_mask_dir)

    @staticmethod
    def read_mask(path:Path):
        mask = skimage.io.imread(path) # type: ignore
        mask = np.where(mask[:, :, 1] == 0, np.uint8(255), np.uint8(0))
        return np.expand_dims(mask, -1)

    def crop(self, window: int):
        self.image_dir.mkdir(exist_ok=True)
        self.mask_dir.mkdir(exist_ok=True)

        cropper = CropUtil()

        for file_name in self.train_files_list:

            cropped_image_view = cropper._crop_one_scene(
                tile_path = self.d_image_dir / file_name,
                window = window,
                read_scene = cropper._read_image
            )

            cropped_mask_view = cropper._crop_one_scene(
                tile_path = self.d_mask_dir / file_name,
                window = window,
                read_scene = self.read_mask
            )

            for image_crop, mask_crop in zip(cropped_image_view, cropped_mask_view):
                crop_name = str(uuid.uuid4())
                cropper._save_as_jpeg_100(image_crop, (self.image_dir / crop_name))
                cropper._save_as_jpeg_100(mask_crop.squeeze(), (self.mask_dir / crop_name))

    async def _download_file(self, session, url:str, file_path:str) -> None:
        async with session.get(url) as response:
                cookies = session.cookie_jar.filter_cookies(url)
                cookie_value = cookies[self.cookie_name].value

        #async with session.get(url, ssl = False) as get_request:
            #cookies = {self.cookie_name: get_request.cookies.get(self.cookie_name)}
        payload = {"csrfmiddlewaretoken": cookie_value, 
                   "password": self.password}

        async with session.post(url+"?dl=1", data = payload) as r:
           async with aiofiles.open(file_path, "wb") as f:
                async for chunk in r.content.iter_any():
                    await f.write(chunk)

    def _extract_vaihingen(self, extractor, low_storage_mode: bool):
        vaihingen_zip_path = self.download_dir / "vaihingen.zip"
        image_dataset_zip_path = self.download_dir / "Vaihingen" / "ISPRS_semantic_labeling_Vaihingen.zip"
        mask_dataset_zip_path = self.download_dir / "Vaihingen" / "ISPRS_semantic_labeling_Vaihingen_ground_truth_COMPLETE.zip"

        extractor.extract_zip_archive(vaihingen_zip_path, self.download_dir, [image_dataset_zip_path.name, mask_dataset_zip_path.name])
        print("Extracted downloaded archives")

        if low_storage_mode:
            #vaihingen_zip_path.unlink()
            print("Deleting downloaded archive to save storage sapce")

        self.d_image_dir.mkdir(exist_ok=True, parents=True)
        self.d_mask_dir.mkdir(exist_ok=True, parents=True)

        extractor.extract_zip_archive(image_dataset_zip_path, image_dataset_zip_path.parent, ["top"])
        extractor.extract_zip_archive(mask_dataset_zip_path, self.d_mask_dir)
        print("Extracted downloaded dataset")

        if low_storage_mode:
            #image_dataset_zip_path.unlink()
            #mask_dataset_zip_path.unlink()
            print("Deleted extracted archives")

        for image_path in (image_dataset_zip_path.parent / "top").glob("*.tif"):
            shutil.move(image_path, self.d_image_dir)
        print("Moved extracted images and masks")

        if low_storage_mode:
            #shutil.rmtree(image_dataset_zip_path.parent)
            pass
    
    def _extract_potsdam(self, extractor, low_storage_mode: bool):
        potsdam_zip_path = self.download_dir / "potsdam.zip"
        images_zip_path = self.download_dir / "Potsdam" / "2_Ortho_RGB.zip"
        masks_zip_path = self.download_dir / "Potsdam" / "5_Labels_all.zip"

        extractor.extract_zip_archive(potsdam_zip_path, self.download_dir, [images_zip_path.name, masks_zip_path.name])
        if low_storage_mode:
            #potsdam_zip_path.unlink()
            print("Deleted downloaded archive")

        images_temp_dir = images_zip_path.parent / images_zip_path.stem
        extractor.extract_zip_archive(images_zip_path, images_zip_path.parent)

        masks_temp_dir = masks_zip_path.parent / masks_zip_path.stem
        (masks_temp_dir).mkdir(exist_ok=True)
        extractor.extract_zip_archive(masks_zip_path, masks_temp_dir)

        print("Extracted images and masks sub archives")

        if low_storage_mode:
            #images_zip_path.unlink()
            #masks_zip_path.unlink()
            print("Deleted extracted sub archives")

        self.d_image_dir.mkdir(exist_ok=True, parents=True)
        self.d_mask_dir.mkdir(exist_ok=True, parents=True)
        #Copy Images, Masks to Correct Directories
        for image_path in images_temp_dir.glob("*.tif"):
            shutil.move(image_path, self.d_image_dir / image_path.name.replace("_RGB", ""))
        for mask_path in masks_temp_dir.glob("*.tif"):
            shutil.move(mask_path, self.d_mask_dir / mask_path.name.replace("_label", ""))
        print("Moved extracted images and masks")

        if low_storage_mode:
            #shutil.rmtree(images_zip_path.parent)
            print("Cleanup directory structure")

    def _get_file_region(self, file_name):
        if "mosaic" in file_name:
            return "vaihingen"
        elif "potsdam" in file_name:
            return "potsdam"
    
    def delete_downloads(self):
        shutil.rmtree(self.download_dir)

    def upload_train_dataset(self, storage: ContaboStorage):
        train_files_list = list(self.mask_dir.glob("*.jpg")) 
        for file_path in tqdm(train_files_list):
            storage.upload_train_pair(file_path.name, self.image_dir, self.mask_dir)

    def upload_test_dataset(self, storage: ContaboStorage):
        test_files_list = list(self.mask_dir.glob("*.tif"))
        for file_path in tqdm(test_files_list):
            storage.upload_test_pair(file_path.name, self.t_image_dir, self.t_mask_dir)

    def catalog_train_dataset(self):
        """Returns a DataFrame with training patch names and dataset name as columns"""

        df = pd.DataFrame({"name": [x.name for x in self.image_dir.rglob("*.jpg")]}) 
        df["dataset"] = [self.root_dir.name]*len(df)
        return df

    def update_train_dataset(self, storage: ContaboStorage):

        #Set path for catalog to be downloaded to 
        #WARINING: training catalog path must be hamed "patches.csv"
        catalog_path = self.root_dir / "metadata" / "patches.csv"

        #If catalog file present in local storage, remove it
        catalog_path.unlink(missing_ok=True)
        catalog_path.parent.mkdir(exist_ok=True, parents=True)

        #Download catalog from bucket
        storage.download_train_catalog(catalog_path)

        #If a catalog is present in storage 
        if catalog_path.exists() and catalog_path.is_file():
            print("Downloaded Catalog Found")
            df = pd.read_csv(catalog_path)

            #Get a view of files associated with current dataset
            dataset_df = df[df["dataset"] == self.root_dir.name]

            #If any files in bucket are associated with the current dataset
            if len(dataset_df) > 0:
               #Remove those files from storage
                print(f"Deleting existing patches in bucket")
                existing_file_names = list(dataset_df.name.apply(lambda x: x))
                for file_name in tqdm(existing_file_names):
                    storage.delete_train_pair(file_name)
                
                #Remove those files from the catalog
                df = df.drop(dataset_df.index, axis = 0)

        
        #If catalog is not present in storage => dataset is not present in storage
        else:  
            #Create a new catalog placeholder
            print("Downloaded Catalog Not Found, Creating New")
            df = pd.DataFrame({"name" : list(), "dataset" : list()})
        
        #Upload training dataset to bucket
        print("Uploading patches to bucket")
        self.upload_train_dataset(storage)
        #Add patches in image, mask dir to catalog
        df = pd.concat([df, self.catalog_train_dataset()], axis = 0)
        #Save catalog to local storage
        df.to_csv(catalog_path, index = False)
        #Upload catalog to bucket
        storage.upload_train_catalog(catalog_path)
        catalog_path.unlink()

In [29]:
DATA = Path("/media/sambhav/30AC4696AC46568E/datasets/urban-feature-extraction")
#DATA = Path("C:/Users/hp/Desktop/datasets/urban-feature-extraction")

In [ ]:
cntb = ContaboStorage(credentials_file = (Path.cwd() / "drive" / "MyDrive" / "aws" / "credentials"),
                      bucket_name = 'building-footprints-dataset')

In [31]:
#cntb = S3StorageEmulator(root = DATA.parent,
                         #credentials_file = (Path.home() / ".aws" / "config"), 
                         #bucket_name= 'building-footprints-dataset')

In [32]:
inria = InriaETL(DATA / "inria")
inria.download()
inria.extract(low_storage_mode=True)
inria.crop(512)
inria.move_test_split()
inria.delete_downloads()
inria.update_train_dataset(cntb) # type: ignore

Training Catalog Not Found
Downloaded Catalog Not Found, Creating New
Uploading patches to bucket


100%|██████████| 15000/15000 [01:51<00:00, 134.71it/s]


In [33]:
cityosm = CityOSMETL(DATA / "city-osm")
cityosm.extract(False)
cityosm.calculate_train_test_split()
cityosm.move_test_split()
cityosm.crop(512)
#cityosm.delete_downloads()
cityosm.update_train_dataset(cntb) # type: ignore

Extracting downloaded archives


100%|██████████| 6/6 [06:40<00:00, 66.68s/it] 


Reorganizing file structure
Downloaded Catalog Found
Uploading patches to bucket


100%|██████████| 62184/62184 [08:34<00:00, 120.78it/s]


In [34]:
isprs = ISPRSETL(DATA / "isprs")
isprs.extract(False)
isprs.calculate_train_test_split()
isprs.move_test_split()
isprs.crop(512)
#isprs.delete_downloads()
isprs.update_train_dataset(cntb) # type: ignore

Extracted downloaded archives
Extracted downloaded dataset
Moved extracted images and masks
Extracted images and masks sub archives
Moved extracted images and masks
Downloaded Catalog Found
Uploading patches to bucket


100%|██████████| 5402/5402 [00:25<00:00, 209.16it/s]


In [ ]:
df = pd.read_csv("C:/Users/hp/Desktop/datasets/building-footprints-dataset/labelled/metadata/patches.csv", index_col=0)
df